In [83]:
# Импорт библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

import time
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from joblib import Parallel, delayed

import cornac
import surprise

| Method                         | RMSE | MAP@K | NDCG@K | Novelty | Diversity |
|--------------------------------|------|-------|--------|---------|-----------|
| User-based                     | 1.05 | 1.00  | 0.87   | 3.84    | 0.79      |
| Item-based                     | 1.01 | 1.00  | 0.88   | 3.87    | 0.92      |
| Clustering                     | 1.04 | 1.00  | 0.87   | 3.95    | 0.80      |
| Clustering + Z-score           | 1.02 | 1.00  | 0.88   | 3.82    | 0.79      |
| KNNregression (classification) | 0.00 | 0.00  | 0.00   | 0.00    | 0.00      |
| MF                             | 0.00 | 0.00  | 0.00   | 0.00    | 0.00      |
| Surprise_MF                    | 0.00 | 0.00  | 0.00   | 0.00    | 0.00      |
| Cornac_NN                      | 0.00 | 0.00  | 0.00   | 0.00    | 0.00      |

User- Item based решения не имеют (имеют 1.00) MAP@K, так как предсказание идет только для конкретных фильмов из test. Иначе предсказание всей матрицы занимает несколько часов из-за поэлементного вычисления



In [106]:
# Функции для метрик
def calculate_rmse(df):
    return np.sqrt(mean_squared_error(df.dropna()['rating'], df.dropna()['predicted_rating']))

def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual: #and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    return score / min(len(actual), k) if len(actual) > 0 else 0.0

def calculate_mapk(df, k=10):
    user_group = df.groupby('userId')
    apk_scores = []
    for user, group in user_group:
        actual = group.dropna()['movieId'].values
        if len(actual) == 0:
            continue
        predicted = group.sort_values(by='predicted_rating', ascending=False)['movieId'].values
        apk_scores.append(apk(actual, predicted, k))
    return np.mean(apk_scores)

def dcg_at_k(r, k):
    r = np.asfarray(r)[:k] # to float
    if r.size:
        return np.sum(r / np.log2(np.arange(2, r.size + 2)))
    return 0.0

def ndcg_at_k(actual, predicted, k=10):
    predicted = predicted[:k]
    ideal = sorted(actual, reverse=True)[:k]
    return dcg_at_k(predicted, k) / dcg_at_k(ideal, k)

def calculate_ndcgk(df, k=10):
    user_group = df.dropna().groupby('userId') # разделить пользователей
    ndcg_scores = []
    for user, group in user_group:
        actual = group['rating'].values # реальный рейтинг
        predicted = group.sort_values(by='predicted_rating', ascending=False)['rating'].values # рейтинг, как решила модель
        ndcg_scores.append(ndcg_at_k(actual, predicted, k))
    return np.mean(ndcg_scores)

def calculate_novelty(df, item_popularity, k=10):
    user_group = df.groupby('userId')
    novelty_scores = []
    for user, group in user_group:
        recommended_items = group.sort_values(by='predicted_rating', ascending=False)['movieId'].values[:k]
        novelty = np.mean([np.log2(len(df['userId'].unique()) / item_popularity[item]) for item in recommended_items if item in item_popularity])
        if not np.isnan(novelty):
            novelty_scores.append(novelty)
    print(rf'$log_2$ max: {np.log2(len(df["userId"].unique())):.3}')
    return np.mean(novelty_scores) if novelty_scores else 0


def calculate_diversity(df, item_similarity_matrix, k=10, matrix_columns=None):
    user_group = df.groupby('userId')
    diversity_scores = []
    
    for user, group in user_group:
        recommended_items = group.sort_values(by='predicted_rating', ascending=False)['movieId'].values[:k]
        total_pairs = 0
        diversity = 0
        for i in range(len(recommended_items)):
            for j in range(i + 1, len(recommended_items)):
                # Получаем индексы фильмов в матрице
                if recommended_items[i] in matrix_columns and recommended_items[j] in matrix_columns:
                    idx_i = matrix_columns.get_loc(recommended_items[i])
                    idx_j = matrix_columns.get_loc(recommended_items[j])
                    total_pairs += 1
                    diversity += (1 - item_similarity_matrix[idx_i, idx_j])
        
        if total_pairs > 0:
            diversity_scores.append(diversity / total_pairs)
    
    return np.mean(diversity_scores) if diversity_scores else 0


In [96]:
def create_cf_matrix(train, cf_type='user'):

    """
    Создает матрицу для коллаборативной фильтрации.
    
    Параметры:
    - train: DataFrame с обучающими данными.
    - cf_type: 'user' для пользователь-фильм матрицы или 'item' для фильм-пользователь матрицы.
    
    Возвращает:
    - Матрица для коллаборативной фильтрации.
    """
    if cf_type == 'user':
        # Матрица пользователь-фильм
        return train.pivot(index='userId', columns='movieId', values='rating')
    elif cf_type == 'item':
        # Матрица фильм-пользователь
        return train.pivot(index='movieId', columns='userId', values='rating')
    else:
        raise ValueError("cf_type должен быть 'user' или 'item'")

In [97]:
def load_and_split_data(file_path, test_size=0.3, random_state=42):
    df = pd.read_csv(file_path)
    train, test = train_test_split(df, test_size=test_size, random_state=random_state)
    return train, test, df

#### Основная часть

In [107]:
# 2-3. User-based and Item-based коллаборативная фильтрация с/без z-score

def collaborative_filtering(train, test, cf_type='user', use_mean_adjustment=False, use_z_score=False, N=50):
    matrix = create_cf_matrix(train, cf_type=cf_type)
    similarity = cosine_similarity(matrix.fillna(0))
    means = matrix.mean(axis=1)
    stds = matrix.fillna(0).std(axis=1) if use_z_score else None

    if use_z_score:
        matrix_normalized = matrix.sub(means, axis=0).div(stds, axis=0).fillna(0)

    # Предварительно вычисляем топ-N для всех пользователей
    top_similarities = np.argsort(-similarity, axis=1)[:, :N]

    def predict_rating(user_id, item_id):
        if cf_type == 'user':
            user_index, item_index = user_id, item_id
        else:
            user_index, item_index = item_id, user_id

        if user_index not in matrix.index or item_index not in matrix.columns:
            return matrix.mean().mean()

        ratings = matrix.loc[user_index]
        similarities = similarity[matrix.index.get_loc(user_index)]
        top_similar = top_similarities[matrix.index.get_loc(user_index)]

        numerator = 0
        denominator = 0
        for similar in top_similar:
            if matrix.iloc[similar][item_index] > 0:
                if use_z_score:
                    adjusted_rating = matrix_normalized.iloc[similar][item_index]
                    numerator += similarities[similar] * adjusted_rating
                else:
                    numerator += similarities[similar] * matrix.iloc[similar][item_index]

                denominator += abs(similarities[similar])

        if denominator == 0:
            return ratings.mean()

        if use_z_score:
            predicted_z_score = numerator / denominator
            return means.loc[user_index] + predicted_z_score * stds.loc[user_index]
        else:
            return numerator / denominator

    test['predicted_rating'] = Parallel(n_jobs=-1)(delayed(predict_rating)(row['userId'], row['movieId']) for _, row in test.iterrows())

    # Вычисляем метрики
    rmse = calculate_rmse(test)
    mapk = calculate_mapk(test, k=10)
    ndcgk = calculate_ndcgk(test, k=10)

    item_popularity = train['movieId'].value_counts().to_dict()
    item_similarity_matrix = 1 - pairwise_distances(matrix.T.fillna(0), metric='cosine')
    novelty = calculate_novelty(test, item_popularity, k=10)
    diversity = calculate_diversity(test, item_similarity_matrix, k=10, matrix_columns=matrix.columns)

    return {
        'RMSE': rmse,
        'MAP@K': mapk,
        'NDCG@K': ndcgk,
        'Novelty': novelty,
        'Diversity': diversity
    }, test

In [110]:
train, test, df = load_and_split_data('data/ml-latest-small/ratings.csv', test_size=0.3)

users = df['userId'].unique()
movies = df['movieId'].unique()
all_combinations = pd.MultiIndex.from_product([users, movies], names=['userId', 'movieId'])
all_combinations_df = pd.DataFrame(index=all_combinations).reset_index()
sampled_df = all_combinations_df.groupby('userId').apply(lambda x: x.sample(n=300, random_state=42))
sampled_df = sampled_df.reset_index(drop=True)
expanded_test = pd.merge(sampled_df, test, on=['userId', 'movieId'], how='left')

C:\Users\romab\AppData\Local\Temp\ipykernel_16684\176393131.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = all_combinations_df.groupby('userId').apply(lambda x: x.sample(n=300, random_state=42))


In [ ]:
user, result = collaborative_filtering(train, expanded_test, cf_type='user', use_mean_adjustment=False, use_z_score=False, N=20)
pd.DataFrame(user, index=[1])

In [24]:
item = collaborative_filtering(train, expanded_test, cf_type='item', use_mean_adjustment=False, use_z_score=False, N=20)
pd.DataFrame(item, index=[1])

$log_2$ max: 9.25


,RMSE,MAP@K,NDCG@K,Novelty,Diversity
1,1.005489,1.0,0.883193,3.866595,0.915602


In [25]:
# 4-5. Рекомендательная система на основе кластеризации и расчета средней оценки кластера
def cluster_based_cf(train, test, n_clusters=40, use_z_score=False, cf_type='user'):
    """
    Кластерная коллаборативная фильтрация.
    
    Параметры:
    - train: DataFrame с обучающими данными.
    - test: DataFrame с тестовыми данными.
    - n_clusters: количество кластеров для K-means.
    - use_z_score: флаг для использования Z-score нормализации.
    - cf_type: 'user' для пользователь-фильм матрицы или 'item' для фильм-пользователь матрицы.
    
    Возвращает:
    - RMSE для предсказанных значений.
    """
    # Создаем матрицу с помощью create_cf_matrix
    matrix = create_cf_matrix(train, cf_type=cf_type)
    
    if use_z_score:
        # Нормализуем оценки с помощью z-score
        means = matrix.mean(axis=1)
        stds = matrix.fillna(0).std(axis=1)
        matrix_normalized = matrix.sub(means, axis=0).div(stds, axis=0)
        matrix_for_clustering = matrix_normalized.fillna(0)
    else:
        # Используем обычную матрицу для кластеризации
        matrix_for_clustering = matrix.fillna(0)
    
    # Применяем K-means кластеризацию
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(matrix_for_clustering)
    
    # Функция для предсказания рейтинга
    def predict_rating(user_id, item_id):
        if cf_type == 'user':
            user_index, item_index = user_id, item_id
        else:
            user_index, item_index = item_id, user_id
        
        # Проверка, если пользователя или элемента нет в матрице
        if user_index not in matrix.index or item_index not in matrix.columns:
            return matrix.mean().mean()
        
        # Определяем кластер пользователя или фильма
        cluster = clusters[matrix.index.get_loc(user_index)]
        cluster_items = matrix.index[clusters == cluster]
        
        if use_z_score:
            # Получаем нормализованные рейтинги для кластера
            cluster_ratings_normalized = matrix_normalized.loc[cluster_items, item_index]
            
            # Проверяем, пусты ли нормализованные рейтинги
            if cluster_ratings_normalized.empty or cluster_ratings_normalized.isna().all():
                return matrix.mean().mean()  # Если нет рейтингов, вернуть среднее по всей матрице
            
            predicted_z_score = cluster_ratings_normalized.mean()
            # Преобразуем предсказание обратно в исходные единицы
            return means[user_index] + (predicted_z_score * stds[user_index])
        else:
            # Получаем обычные рейтинги для кластера
            cluster_ratings = matrix.loc[cluster_items, item_index]
            
            # Проверка на наличие не NaN значений
            if cluster_ratings.empty or cluster_ratings.isna().all():
                return matrix.mean().mean()  # Если нет доступных значений, вернуть среднее по всей матрице
            
            # Возвращаем среднее значение по кластеру
            return cluster_ratings.mean()
    
    # Предсказываем рейтинги для тестового набора
    test['predicted_rating'] = test.apply(lambda row: predict_rating(row['userId'], row['movieId']), axis=1)
    
    # Вычисляем метрики
    rmse = calculate_rmse(test)
    mapk = calculate_mapk(test, k=10)
    ndcgk = calculate_ndcgk(test, k=10)

    item_popularity = train['movieId'].value_counts().to_dict()
    item_similarity_matrix = 1 - pairwise_distances(matrix.T.fillna(0), metric='cosine')
    novelty = calculate_novelty(test, item_popularity, k=10)
    diversity = calculate_diversity(test, item_similarity_matrix, k=10, matrix_columns=matrix.columns)

    return {
        'RMSE': rmse,
        'MAP@K': mapk,
        'NDCG@K': ndcgk,
        'Novelty': novelty,
        'Diversity': diversity
    }

In [26]:
clustering = cluster_based_cf(train, test, n_clusters=40, use_z_score=False, cf_type='user')
pd.DataFrame(clustering, index=[1])

$log_2$ max: 9.25


,RMSE,MAP@K,NDCG@K,Novelty,Diversity
1,1.04326,1.0,0.865875,3.945203,0.798151


In [27]:
clustering = cluster_based_cf(train, test, n_clusters=40, use_z_score=True, cf_type='user')
pd.DataFrame(clustering, index=[1])

$log_2$ max: 9.25


,RMSE,MAP@K,NDCG@K,Novelty,Diversity
1,1.02404,1.0,0.87568,3.821923,0.79185


In [28]:
# 6. Рекомендательная система на основе голосования
def voting_based_cf(train, test, cf_type='user', n_neighbors=40, use_knn=False):
    """
    Коллаборативная фильтрация с голосованием на основе сходства и KNN.

    Параметры:
    - train: DataFrame с обучающими данными.
    - test: DataFrame с тестовыми данными.
    - cf_type: 'user' для пользователь-фильм матрицы или 'item' для фильм-пользователь матрицы.
    - n_neighbors: количество соседей для KNN или топ-N для голосования.
    - use_knn: флаг для использования KNN вместо простого выбора топ-N похожих пользователей/фильмов.

    Возвращает:
    - RMSE для предсказанных значений.
    """
    # Создаем матрицу для коллаборативной фильтрации
    matrix = create_cf_matrix(train, cf_type=cf_type)
    
    # Если используем KNN
    if use_knn:
        knn = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine').fit(matrix.fillna(0))
    
    # Вычисляем косинусное сходство, если не используем KNN
    similarity = cosine_similarity(matrix.fillna(0)) if not use_knn else None

    # Функция для предсказания рейтинга
    def predict_rating(user_id, item_id):
        if cf_type == 'user':
            user_index, item_index = user_id, item_id
        else:
            user_index, item_index = item_id, user_id

        # Если пользователя или элемента нет в матрице
        if user_index not in matrix.index or item_index not in matrix.columns:
            return matrix.mean().mean()

        # Если используем KNN
        if use_knn:
            # Найти ближайших соседей для пользователя
            distances, neighbors = knn.kneighbors([matrix.fillna(0).loc[user_index]], n_neighbors=n_neighbors)
            neighbors = neighbors.flatten()  # Индексы соседей
        else:
            # Вычисляем схожесть и выбираем топ-N похожих пользователей/фильмов
            similarities = similarity[matrix.index.get_loc(user_index)]
            neighbors = np.argsort(similarities)[-n_neighbors-1:-1]  # Топ-N соседей

        # Собираем рейтинги соседей для указанного элемента
        neighbor_ratings = matrix.iloc[neighbors][item_index].values
        valid_ratings = neighbor_ratings[neighbor_ratings > 0]

        # Если нет валидных рейтингов, возвращаем средний рейтинг пользователя
        if len(valid_ratings) == 0:
            return matrix.loc[user_index].mean()

        # Возвращаем среднее значение рейтингов соседей
        return valid_ratings.mean()

    # Предсказываем рейтинги для тестового набора
    test['predicted_rating'] = test.apply(lambda row: predict_rating(row['userId'], row['movieId']), axis=1)
    
    # Вычисляем метрики
    rmse = calculate_rmse(test)
    mapk = calculate_mapk(test, k=10)
    ndcgk = calculate_ndcgk(test, k=10)

    item_popularity = train['movieId'].value_counts().to_dict()
    item_similarity_matrix = 1 - pairwise_distances(matrix.T.fillna(0), metric='cosine')
    novelty = calculate_novelty(test, item_popularity, k=10)
    diversity = calculate_diversity(test, item_similarity_matrix, k=10, matrix_columns=matrix.columns)

    return {
        'RMSE': rmse,
        'MAP@K': mapk,
        'NDCG@K': ndcgk,
        'Novelty': novelty,
        'Diversity': diversity
    }

In [29]:
knn = voting_based_cf(train, test, cf_type='user', n_neighbors=40, use_knn=True)
pd.DataFrame(knn, index=[1])

KeyboardInterrupt: 